## Criação das planilhas CSV para importação

Agora que as planilhas fictícias foram geradas e o tratamento na planilha de salário/cargo feito, é possível pensar no problema e começar a criação dos arquivos CSV para o time de TI completar a importação. 

### Funcionários

Cada funcionário precisa receber um ID que vai ser utilizado em quase todas as outras planilhas, afim de identificação. Então vou chamar o primeiro CSV de "funcionarios" e ele vai ser salvo na pasta "output".

In [1]:
'''Importando todos os pacotes'''
import warnings
import os
from datetime import date
import numpy as np
import pandas as pd
import uuid

pd.set_option('display.max_columns', 40) # Exibe todas as colunas
# Ignora o seguinte alerta do Pandas
# FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
warnings.simplefilter(action='ignore', category=FutureWarning)

'''
Como vou precisar do ID não só para cada funcionário, mas também um ID para cada planilha,
e possivelmente para cada cargo, a lógica vai sempre se repetir.
Por essa razão é interessante criar uma função. 

'''

def get_uuid4(
    dataframe:pd.DataFrame,
    check_column_name:str,
    fill_column_name:str)->pd.DataFrame:
    '''
    Usa o UUID para gerar uma identificação aleatória.
    O ID do funcionário é único para cada funcionário, 
    este ID é uma referência para outras planilhas.
    
    1. Esta função precisa passar o DataFrame onde os IDs serão gerados.
    2. Nome da coluna que é usada como parâmetro do loop.
    3. Nome da coluna onde os IDs gerados serão contidos.
    '''
    for inx, rows in dataframe.iterrows():
        if rows[check_column_name] != '':
            dataframe.loc[inx, fill_column_name] = uuid.uuid4()

In [2]:
data = pd.read_excel("planilhas\\rh_main.xlsx")

# Cria duas novas colunas no dataframe, para preencher com os IDs.
data['employee_id'] = '' 
data['leader_id'] = ''

# Também será criado duas colunas novas contendo a data que o script for rodado.
# Garante a diferenciação dos funcinários importados e dos novos registros feitos no novo sistema.
today = date.today().strftime("%m-%d-%Y")

data.replace('',np.nan,inplace=True)

# Usa a função criada, para atribuir o ID na coluna nova
get_uuid4(data,'Nome Funcionário','employee_id')

#--------------------Gerando o CSV de Employee --------------------#
employee_sheet = pd.DataFrame({'id': data['employee_id'],
                            'Nome': data['Nome Funcionário'],
                            'Telefone': data['Celular'],
                            'Email': data['E-Mail'],
                            'Situação': data['Situação'],
                            'Criado_em':today,
                            'Upload_em':today})

# Save the new spreadsheet
employee_sheet.to_csv('output\\funcionarios.csv', index=False)

employee_sheet.head(5)


,id,Nome,Telefone,Email,Situação,Criado_em,Upload_em
0,a16a156e-aa4a-40c5-94a4-4bea0f443f39,Lucas Gabriel da Rosa,+55 (081) 0623-4078,viniciussales@example.com,Ativo,01-30-2023,01-30-2023
1,727208ef-44e7-4c85-8c25-199f7011e9f5,Clarice Almeida,(021) 1104-9825,vrezende@example.net,Ativo,01-30-2023,01-30-2023
2,ad64f750-8850-444b-9bb1-536951a1819d,João Lucas Cunha,+55 11 5068 0895,isabellapeixoto@example.net,Ativo,01-30-2023,01-30-2023
3,945e364e-0bc3-4ec7-a693-32fc0f8db8a1,Cecília Fogaça,(061) 7064-1466,luizacosta@example.net,Ativo,01-30-2023,01-30-2023
4,f5fcda1f-5ef0-4e87-9768-70e048f232ae,Isadora Cavalcanti,+55 71 5348-5632,caio52@example.com,Ativo,01-30-2023,01-30-2023


### Dados pessoais

Agora com a planilha de funcionarios gerada, podemos criar a próxima que contém os dados pessoais de cada empregado. Vou utilizar o **pandas.merge** aqui, porque existem dados pessoais em mais de uma planilha. 

O merge serve para unir duas planilhas escolhendo uma coluna de igualdade. O parâmetro 'on' define que o merge vai acontecer de acordo com a coluna escolhida. Em outras palavras ele vai realizar uma junção parecida com PROCV do Excel.

Se tiver um macth nessa coluna o pandas vai trazer os dados das demais. Com o parâmetro 'how=left' o dataframe da esquerda vai ser mantido, e as informações do dataframde da direita vão fazer parte do df da esquerda, caso eu quisse o contrário poderia passar how=right.

In [3]:
# Além da planilha principal vamos usar outra planilha, a de raça e cor. 
raca_cor = pd.read_excel('planilhas\\raca_cor_dependentes.xlsx')

# Chamando método melt
data_raca = data.merge(raca_cor,
                    on='Nome Funcionário',
                    how='left')

#--------------------Generating the personalData Spreadsheet--------------------#
personaldata_sheet = pd.DataFrame({'employee_id': data_raca['employee_id'],
                                'CPF': data_raca['CPF'],                        
                                'Nome': data_raca['Nome Funcionário'],
                                'Email': data_raca['E-Mail'],
                                'Telefone': data_raca['Celular'],
                                'RG': data_raca['RG'],
                                'Nacionalidade': data_raca['Nacionalidade'],
                                'Dt. Nascimento': data_raca['Dt. Nascimento'],
                                'Sexo': data_raca['Sexo'],
                                'Raça e Cor': data_raca['Raça/Cor'],
                                'Estado Civil': data_raca['Est. Civil'],
                                'Mãe': data_raca['Mãe'],
                                'Pai': data_raca['Pai'],
                                'Endereço Completo': data_raca['Endereço'],
                                'Criado_em': today,
                                'Upload_em': today})


personaldata_sheet.insert(0,'id','') #Creat the new column on dataframe

get_uuid4(personaldata_sheet,'employee_id','id') # Use the function to generate the ID

# Save the new spreadsheet
personaldata_sheet.to_csv('output\\dados_pessoais.csv', index=False)

personaldata_sheet.head(5)

,id,employee_id,CPF,Nome,Email,Telefone,RG,Nacionalidade,Dt. Nascimento,Sexo,Raça e Cor,Estado Civil,Mãe,Pai,Endereço Completo,Criado_em,Upload_em
0,effbe3ff-c530-4872-a73d-cba6a4f7e785,a16a156e-aa4a-40c5-94a4-4bea0f443f39,42116450377,Lucas Gabriel da Rosa,viniciussales@example.com,+55 (081) 0623-4078,59530754,Brasileiro(a),1972-05-19,Feminino,Branco(a),Casado(a),Srta. Juliana da Conceição,Thomas Teixeira,"Esplanada Novaes, 17\nDona Clara\n14010-821 Cu...",01-30-2023,01-30-2023
1,c9f4e84e-b16d-4820-aa56-1febbabddc29,727208ef-44e7-4c85-8c25-199f7011e9f5,78286620035,Clarice Almeida,vrezende@example.net,(021) 1104-9825,58917100,Brasileiro(a),2018-02-21,Feminino,Branco(a),Viúvo(a),Maitê da Cruz,Igor da Cruz,Ladeira de Gonçalves\nVila Fumec\n00081-863 Ca...,01-30-2023,01-30-2023
2,29d0d7b5-8ce0-485d-8de1-11518ca36ba6,ad64f750-8850-444b-9bb1-536951a1819d,71222935371,João Lucas Cunha,isabellapeixoto@example.net,+55 11 5068 0895,23612766,Brasileiro(a),2005-04-28,Feminino,Preto(a),Casado(a),Yasmin Aragão,Matheus Vieira,"Residencial Luiz Henrique Ramos, 124\nVila Nov...",01-30-2023,01-30-2023
3,f0ccc531-3922-4f94-867e-175d6a786ffe,945e364e-0bc3-4ec7-a693-32fc0f8db8a1,37308822484,Cecília Fogaça,luizacosta@example.net,(061) 7064-1466,69434628,Brasileiro(a),1987-02-06,Masculino,Preto(a),Casado(a),Esther Souza,Sr. Isaac da Cruz,"Praça de Rezende, 38\nCaiçaras\n61521803 Araúj...",01-30-2023,01-30-2023
4,589a0708-dbf6-4bb6-8211-878f6a299474,f5fcda1f-5ef0-4e87-9768-70e048f232ae,22628655033,Isadora Cavalcanti,caio52@example.com,+55 71 5348-5632,49289584,Brasileiro(a),2013-02-14,Feminino,Preto(a),Divorciado(a),Maria Alice Costela,Vicente Duarte,"Praia de Souza, 19\nCentro\n81589-994 da Mota ...",01-30-2023,01-30-2023


### Dependentes

Na planilha anterior, com os dados pessoais, também constam informações dos dependêntes. Nesse caso se chamar **data_raca** essa informações de cada um dos funcionários também vão estar presentes. Para gerar esse CSV vou usar **pd.melt**. Na planilha "raca_cor_dependentes", criada anteriormente, possui uma organização onde cada linha representa um único funcionário, e contém todas as demais informações. 

![raca_cor](imagens/raca_cor.JPG)

O primeiro funcionário, Lucas Gabriel da Rosa então tem três dependentes. Mas não quero que essa planilha mantenha essa organização. Para que seja importada corretamente, preciso que o CSV tenha somente uma coluna de dependentes, conforme a imagem abaixo:

![raca_cor_ideal](imagens/raca_cor_ideal.JPG)

E é exatamente isso que o método melt irá fazer. Vai organizar o dataframe saindo de um formato amplo para um longo. Depois de usar o pd.melt e agrupar o trio de colunas e uma. Vou utilizar o concat do pandas para unir essas três novas colunas (contendo todas as informações dos dependentes) no novo CSV.

In [4]:
#--------------------Gerando CSV de Dependentes--------------------#
# Usando o pandas melt para mudar a disposição do dataframe
# Perceba que a união dessas colunas vai ser pelo employee_id, é a variável indicadora. 
dependenttype_column = pd.melt(data_raca.reset_index(),
                            id_vars=['employee_id'],
                            value_vars=['Parentesco Dependente 1',
                                        'Parentesco Dependente 2',
                                        'Parentesco Dependente 3'],
                            value_name='Parentesco Dependente').drop('variable',1)

cpf_dependent = pd.melt(data_raca.reset_index(),
                    id_vars=['employee_id'],
                    value_vars=['CPF Dependente 1',
                                'CPF Dependente 2',
                                'CPF Dependente 3'],
                    value_name='CPF Dependente').drop('variable',1)

name_dependet = pd.melt(data_raca.reset_index(),
                    id_vars=['employee_id'],
                    value_vars=['Nome Dependente 1',
                                'Nome Dependente 2',
                                'Nome Dependente 3'],
                    value_name='Nome Dependentes').drop('variable',1)

birthdate_dependet = pd.melt(data_raca.reset_index(),
                    id_vars = ['employee_id'],
                    value_vars = ['Nascimento Dependente 1',
                                'Nascimento Dependente 2',
                                'Nascimento Dependente 3'],
                    value_name='Dt. Nascimento Dependentes').drop('variable',1)

# 
dependents_sheet = (pd.concat([name_dependet,
                                cpf_dependent['CPF Dependente'],
                                dependenttype_column['Parentesco Dependente'],
                                birthdate_dependet['Dt. Nascimento Dependentes'],],axis=1).fillna(''))

dependents_sheet.insert(0,'id','')# Criando a coluna nova de ID
dependents_sheet.insert(3,'Possui Dependente?','')

dependents_sheet['Criado_em'] = today
dependents_sheet['Upload_em'] = today

# Preenche a nova coluna "Possui Dependete?" baseado na coluna com os nomes.
# Se tiver um nome nessa coluna então o funcionário tem dependente. 
for indicator, line in dependents_sheet.iterrows():
    if line['Nome Dependentes'] != '':
        dependents_sheet.loc[indicator, 'Possui Dependente?'] = True
    else:
        dependents_sheet.loc[indicator, 'Possui Dependente?'] = False

get_uuid4(dependents_sheet,'employee_id','id') # Obtem o ID do CSV


# Save the new spreadsheet
dependents_sheet.to_csv('output\\dependentes.csv',index=False,encoding='utf-8')


dependents_sheet.head(5)

,id,employee_id,Nome Dependentes,Possui Dependente?,CPF Dependente,Parentesco Dependente,Dt. Nascimento Dependentes,Criado_em,Upload_em
0,7ea78c49-4118-4dfd-8a49-785de09eb014,a16a156e-aa4a-40c5-94a4-4bea0f443f39,Gabrielly Lopes,True,40784258520.0,Filho(a) menos de 21 anos,1979-10-31,01-30-2023,01-30-2023
1,d741fb15-c1d3-4160-9149-2fb77d18918e,727208ef-44e7-4c85-8c25-199f7011e9f5,Sophie Nunes,True,88215581447.0,Filho(a) mais de 21 anos,2003-07-04,01-30-2023,01-30-2023
2,98e497fe-d1a0-4218-b591-c8d7a078b15e,ad64f750-8850-444b-9bb1-536951a1819d,Cecília Duarte,True,33383153282.0,Filho(a) mais de 21 anos,2019-07-25,01-30-2023,01-30-2023
3,393a1390-c358-44c4-8f72-b9fcec33cfbb,945e364e-0bc3-4ec7-a693-32fc0f8db8a1,Matheus Rezende,True,78384204184.0,Filho(a) menos de 21 anos,2004-05-18,01-30-2023,01-30-2023
4,ddd28bec-0e51-4c3b-adca-8b24fdf6293e,f5fcda1f-5ef0-4e87-9768-70e048f232ae,Lavínia da Mota,True,90655319469.0,Filho(a) menos de 21 anos,2021-01-03,01-30-2023,01-30-2023


### Férias

Vou trabalhar agora com uma planilha nova, nela contém informações sobre o período de férias de alguns funcionários e os dias gozados. Porém antes de partir para criação do CSV essa planilha vai precisar passar por um tratamento de dados. Ela está cheia de formatações, assim quando o pandas lê a planilha tenta encaixar todas as informações em um padrão. Então o tratamento aqui consiste em remover colunas, linhas e caracteres indesejáveis.

In [5]:
# Vou carregar a planilha e mostrar o resultado quando chamamos com o head
df = pd.read_excel('planilhas\\historico_ferias.xls')

df.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PROGRAMAÇÃO DE FÉRIAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Data,Vencto.,NaN,Fer.,NaN,NaN,Fer.,Início,Fim,NaN,NaN,NaN,NaN,Início,NaN,NaN,NaN,NaN,NaN,Dias,Dias,NaN,NaN,Dias,Limite,NaN,Dias,NaN,Dias
5,Código,NaN,Empregado,NaN,NaN,NaN,NaN,NaN,NaN,admissão,férias,venc.,NaN,NaN,NaN,pro.,aquisitivo,aquisitivo,NaN,NaN,NaN,NaN,gozo férias,Dias,NaN,Abono,NaN,13º,dir.,goz.,NaN,NaN,rest.,p/ gozo,NaN,afast.,NaN,faltas
6,74,NaN,Caroline da Mota,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-01 00:00:00,2022-02-28 00:00:00,NaN,NaN,1,10/12,NaN,2021-03-01 00:00:00,2022-02-28 00:00:00,NaN,NaN,NaN,NaN,..../..../......,....,....,NaN,NaN,....,30,NaN,24.0,NaN,6,2023-02-23 00:00:00,NaN,-,NaN,-
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-01 00:00:00,2023-02-28 00:00:00,NaN,NaN,NaN,NaN,..../..../......,....,....,NaN,NaN,....,25,NaN,0.0,NaN,30,2024-01-30 00:00:00,NaN,-,NaN,-
8,80,NaN,Arthur Teixeira,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-15 00:00:00,2022-03-14 00:00:00,NaN,NaN,1,10/12,NaN,2021-03-15 00:00:00,2022-03-14 00:00:00,NaN,NaN,NaN,NaN,..../..../......,....,....,NaN,NaN,....,30,NaN,15.0,NaN,5,2023-03-10 00:00:00,NaN,-,NaN,-
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-15 00:00:00,2023-03-14 00:00:00,NaN,NaN,NaN,NaN,..../..../......,....,....,NaN,NaN,....,25,NaN,0.0,NaN,30,2024-02-14 00:00:00,NaN,-,NaN,-


In [6]:
# Então para facilitar o tratamento de dados vou converter essa planilha em CSV.
df.to_csv('planilhas\\ferias_temp.csv',index=False, header=False)

ferias = pd.read_csv('planilhas\\ferias_temp.csv',header=None)

# Usando o drop para deletar as linhas e colunas desnecessárias.
ferias.drop([0,1,2,3,4,5], inplace=True)
ferias.drop(ferias.tail(3).index,inplace=True) # Combinando o drop com o tail eu consigo sempre excluir as ultimas linhas, não importa o conteúdo dessas
ferias.drop([0,1,3,4,5,6,7,8,11,
                    12,15,18,19,20,22,23,
                    21,24,25,26,31,36,34,
                    27,29,35,37],axis=1, inplace=True)

ferias.head(5)

,2,9,10,13,14,16,17,28,30,32,33
6,Caroline da Mota,2021-03-01 00:00:00,2022-02-28 00:00:00,1,10/12,2021-03-01 00:00:00,2022-02-28 00:00:00,30,24.0,6,2023-02-23 00:00:00
7,NaN,NaN,NaN,NaN,NaN,2022-03-01 00:00:00,2023-02-28 00:00:00,25,0.0,30,2024-01-30 00:00:00
8,Arthur Teixeira,2021-03-15 00:00:00,2022-03-14 00:00:00,1,10/12,2021-03-15 00:00:00,2022-03-14 00:00:00,30,15.0,5,2023-03-10 00:00:00
9,NaN,NaN,NaN,NaN,NaN,2022-03-15 00:00:00,2023-03-14 00:00:00,25,0.0,30,2024-02-14 00:00:00
10,Matheus Melo,2021-03-15 00:00:00,2022-03-14 00:00:00,1,10/12,2021-03-15 00:00:00,2022-03-14 00:00:00,30,22.0,8,2023-03-07 00:00:00


Agora já temos um resultado mais amigável e limpo, próximo passo é renomear as colunas. Vou usar **pd.rename**. Cada coluna recebeu como cabeçalho sua posição no index, vou alterar esses números por nomes, que fazem mais sentido. Também vou passar o parâmetro **"inplace=True"** (que por padrão é False). O inplace faz com que as alterações realizadas no dataframe sejam salvas, ou seja minha variável "ferias" vai exibir o DF com as alterações feitas e não mais o DF com os números no lugar do cabeçalho. 

In [7]:
# Usando o pd.rename e passando o inplace para salvar

ferias.rename(columns={2:'Nome Funcionário',
                    9: 'Data Admissão',
                    10: 'Vencimento Férias',
                    13: 'Fer. Venc.',
                    14: 'Fer. Pro.',
                    16: 'Início aquisitivo',
                    17: 'Fim aquisitivo',
                    33: 'Limite p/ gozo',
                    30: 'Dias gozados',
                    28: 'Dias dir.',
                    32: 'Dias restantes'
                    },inplace=True)

ferias.head(5)

,Nome Funcionário,Data Admissão,Vencimento Férias,Fer. Venc.,Fer. Pro.,Início aquisitivo,Fim aquisitivo,Dias dir.,Dias gozados,Dias restantes,Limite p/ gozo
6,Caroline da Mota,2021-03-01 00:00:00,2022-02-28 00:00:00,1,10/12,2021-03-01 00:00:00,2022-02-28 00:00:00,30,24.0,6,2023-02-23 00:00:00
7,NaN,NaN,NaN,NaN,NaN,2022-03-01 00:00:00,2023-02-28 00:00:00,25,0.0,30,2024-01-30 00:00:00
8,Arthur Teixeira,2021-03-15 00:00:00,2022-03-14 00:00:00,1,10/12,2021-03-15 00:00:00,2022-03-14 00:00:00,30,15.0,5,2023-03-10 00:00:00
9,NaN,NaN,NaN,NaN,NaN,2022-03-15 00:00:00,2023-03-14 00:00:00,25,0.0,30,2024-02-14 00:00:00
10,Matheus Melo,2021-03-15 00:00:00,2022-03-14 00:00:00,1,10/12,2021-03-15 00:00:00,2022-03-14 00:00:00,30,22.0,8,2023-03-07 00:00:00


Alguns funcionário possuem mais de um perído de férias. Quando isso ocorre na planilha original, gera uma linha em branco. 

![ferias](imagens/ferias.JPG)

A funcionária fictícia Caroline da Mota tem direito a 30 dias de férias, mas decidiu não usar todos de uma vez. Os dias faltantes então geraram uma outra linha em branco (em azul). Isso faz com que o dataframe aparece com valores NaN (missing). Não tem como ter certeza qual o vencimento das férias seguintes, portanto a coluna "Vencimento de Férias" deve continuar com valor NaN. Mas as outras duas colunas, são valores que não se alteram ao longo do tempo, então posso dizer que o valor NaN é exatamente igual ao valor acima dele. Para contornar esse problema existe um método do pandas que faz exatamente o que eu preciso, chama **fillna**. Que quer dizer preencha o valor NaN.

In [8]:
'''
Para que o fillna funcione corretamente, eu preciso dizer ao pandas com o que ele deve preencher os valores missing.
No meu exmeplo eu preciso que ele preencha com o valor da célula acima e é exatamente isso que o parâmetro "method='ffill'" faz.
Também vou passar o parâmetro axis=0 para fazer o processo por linhas e inplace para salvar na mesma variável o resultado.
'''
ferias['Nome Funcionário'].fillna(axis=0,method='ffill',inplace=True)
ferias['Data Admissão'].fillna(axis=0,method='ffill',inplace=True)

ferias.head(5) # Chamando o novo dataframe

,Nome Funcionário,Data Admissão,Vencimento Férias,Fer. Venc.,Fer. Pro.,Início aquisitivo,Fim aquisitivo,Dias dir.,Dias gozados,Dias restantes,Limite p/ gozo
6,Caroline da Mota,2021-03-01 00:00:00,2022-02-28 00:00:00,1,10/12,2021-03-01 00:00:00,2022-02-28 00:00:00,30,24.0,6,2023-02-23 00:00:00
7,Caroline da Mota,2021-03-01 00:00:00,NaN,NaN,NaN,2022-03-01 00:00:00,2023-02-28 00:00:00,25,0.0,30,2024-01-30 00:00:00
8,Arthur Teixeira,2021-03-15 00:00:00,2022-03-14 00:00:00,1,10/12,2021-03-15 00:00:00,2022-03-14 00:00:00,30,15.0,5,2023-03-10 00:00:00
9,Arthur Teixeira,2021-03-15 00:00:00,NaN,NaN,NaN,2022-03-15 00:00:00,2023-03-14 00:00:00,25,0.0,30,2024-02-14 00:00:00
10,Matheus Melo,2021-03-15 00:00:00,2022-03-14 00:00:00,1,10/12,2021-03-15 00:00:00,2022-03-14 00:00:00,30,22.0,8,2023-03-07 00:00:00


O dataframe já está quase pronto. Agora olhando para as colunas que contém data, o pandas está identificando como do tipo objeto. 

![data](imagens/dtype_data.JPG)

Não quero que fique dessa forma, pois caso haja necessidade de trabalhar com data é ideal que estaja no formato correto que é datetime64

In [9]:
#  Para corrigir o formato da data posso passar o método 'astype' 
ferias['Data Admissão'] = ferias['Início aquisitivo'].astype('datetime64')
ferias['Início aquisitivo'] = ferias['Início aquisitivo'].astype('datetime64')
ferias['Fim aquisitivo'] = ferias['Fim aquisitivo'].astype('datetime64')
ferias['Limite p/ gozo'] = ferias['Limite p/ gozo'].astype('datetime64')

# print(ferias['Data Admissão'].dtype)

In [10]:
# Agora com o dataframe organizado e limpo é possível criar o CSV
# Criar o CSV de ferias usando a mesma estrutura lógicas das demais planilhas

'''
Note que para esse pd.merge eu preciso alterar o parâmetro 'how'.
Agora eu não quero manter a planilha principal, quero manter a nova planilha.
Isso porque nem todos os funcionários são CLT e possuem direito a férias. Ou ainda não completaram um ano.
Poucos funcionários possuem férias, e só quero que esses apareçam no CSV. 
'''
data_ferias = data.merge(ferias,
                        on='Nome Funcionário',
                        how='right')

#--------------------Gerando o CSV de férias--------------------#
acquisition_sheet = pd.DataFrame({'employee_id': data_ferias['employee_id'],
                                'Name': data_ferias['Nome Funcionário'],
                                'Início do período': data_ferias['Início aquisitivo'],
                                'Fim do período': data_ferias['Fim aquisitivo'],
                                'Limite p/ gozo': data_ferias['Limite p/ gozo'],
                                'Status': True,
                                'Criado_em': today,
                                'Upload_em': today})


acquisition_sheet.insert(0,'id','') # Criando nova coluna para receber o ID

get_uuid4(acquisition_sheet,'employee_id','id') # Gerando o ID

# Salvando o nome CSV
acquisition_sheet.to_csv('output\\ferias.csv', index=False)

# Apagando o arquivo temporário criado 
if os.path.exists('planilhas\\ferias_temp.csv'):
    os.remove('planilhas\\ferias_temp.csv')

acquisition_sheet.head(5)

,id,employee_id,Name,Início do período,Fim do período,Limite p/ gozo,Status,Criado_em,Upload_em
0,7c6d232e-9654-4707-b9a4-437a71f129d3,bc044df8-4bac-45e4-a0c1-efe22bc6470c,Caroline da Mota,2021-03-01,2022-02-28,2023-02-23,True,01-30-2023,01-30-2023
1,429abf2d-36a4-480a-9c7e-bdca17f8a150,bc044df8-4bac-45e4-a0c1-efe22bc6470c,Caroline da Mota,2022-03-01,2023-02-28,2024-01-30,True,01-30-2023,01-30-2023
2,95648dc5-1d3e-4bad-bf98-531aaffdbf65,c723f79d-dcc1-4c1c-9570-5991a862ec79,Arthur Teixeira,2021-03-15,2022-03-14,2023-03-10,True,01-30-2023,01-30-2023
3,51d92796-79cd-4186-9e34-331f5f81bd50,c723f79d-dcc1-4c1c-9570-5991a862ec79,Arthur Teixeira,2022-03-15,2023-03-14,2024-02-14,True,01-30-2023,01-30-2023
4,853d4b0f-09b3-4590-8f79-6b9fccf31780,5fbe8ff8-76fc-40b1-bd2d-db2d0733f591,Matheus Melo,2021-03-15,2022-03-14,2023-03-07,True,01-30-2023,01-30-2023


### Histórico de Salário

Depois de ter feito o tratamento de dados, que você pode conferir abrindo o arquivo [tratamendo_dados_salario_cargo](tratamento_dados_salario_cargo.ipynb) é possível preparar o CSV que vai ter as informações sobre as alterações de salário. Porém ainda tem mais um pequeno desafio no caminho. 

Não é um problema não ter uma data final para um salário nem para a troca de cargo, isso porque só vai existir uma data final caso tenha sido desligado da empresa ou tenha mudado de salário novamente. Nesse caso a coluna "Final" que contem as datas é normal e esperado que tenha alguns valores missing. Mas isso não deve acontecer para as datas Iniciais. Na planilha anterior que foi feita o tratamento não consta essa informação. Sabemos que toda data inicial tanto para primeiro salário quanto para o primeiro cargo é a data de admissão. Essa informação tem na planilha principal do RH e vou usar merge, para completar a planilha agora, antes de criar o CSV. 

In [11]:
# Carregando o csv temporário criado no tratamento de dados
salario = pd.read_csv('planilhas\\salario_temp_historico.csv',sep = ',')

data_salario = data.merge(salario,
                        on='Nome Funcionário',
                        how='left')

#--------------------Gerando o CSV de Salário--------------------#
# Aqui está a correção do nosso problema mencionado acima. 
# Se a data inicial for fazia eu completo pela data de admissão. O fillna do pandas é ideial!
data_salario['Inicial'].fillna(data_salario['Dt. Admissão'],inplace=True)


salary_sheet = pd.DataFrame({'employee_id': data_salario['employee_id'],
                            'Salários': data_salario['Salário_y'],
                            'Data Inicial': data_salario['Inicial'],
                            'Data FInal': data_salario['Final'],
                            'Bonus': data_salario['Adcional'],
                            'Criado_em': today,
                            'Upload_em': today})

salary_sheet.insert(0,'id','') # Cria nova coluna no dataframe

get_uuid4(salary_sheet,'employee_id','id')

# Salva nova planilha
salary_sheet.to_csv('output\\salario.csv', index=False)

salary_sheet.head(10)

,id,employee_id,Salários,Data Inicial,Data FInal,Bonus,Criado_em,Upload_em
0,0ef746d9-7533-4091-819c-8af30c02c151,a16a156e-aa4a-40c5-94a4-4bea0f443f39,"2100,00",2014-05-23,2022-04-01,NaN,01-30-2023,01-30-2023
1,e26301c2-e9f9-4479-b04a-fd19de94eeda,a16a156e-aa4a-40c5-94a4-4bea0f443f39,"2186,31",2022-04-01,2022-05-01,NaN,01-30-2023,01-30-2023
2,afea7dc9-a7d8-4301-8b5d-6a94b4bfb0a1,a16a156e-aa4a-40c5-94a4-4bea0f443f39,"2276,40",2022-05-01,NaN,NaN,01-30-2023,01-30-2023
3,23b10056-5d58-4373-bb2a-c9202de33c49,727208ef-44e7-4c85-8c25-199f7011e9f5,"2740,00",1981-10-27,2022-04-01,NaN,01-30-2023,01-30-2023
4,2ca0661e-434f-49e9-a084-a89c4ae1f671,727208ef-44e7-4c85-8c25-199f7011e9f5,"2922,48",2022-04-01,NaN,NaN,01-30-2023,01-30-2023
5,7a9d9d0b-8ec4-4d6a-a168-4a9029fc854e,ad64f750-8850-444b-9bb1-536951a1819d,"2100,00",1993-06-25,2022-05-01,NaN,01-30-2023,01-30-2023
6,8c26857e-5447-4af5-b767-a4aedc66a340,ad64f750-8850-444b-9bb1-536951a1819d,"2276,40",2022-05-01,NaN,NaN,01-30-2023,01-30-2023
7,f9259090-d13e-407d-ac75-6b0ab8d6f3c8,945e364e-0bc3-4ec7-a693-32fc0f8db8a1,"2100,00",2006-02-02,2022-04-01,NaN,01-30-2023,01-30-2023
8,3fe8e8fe-b7e1-4939-8ba3-71214de0805a,945e364e-0bc3-4ec7-a693-32fc0f8db8a1,"2186,31",2022-04-01,2022-04-02,NaN,01-30-2023,01-30-2023
9,f235cda6-ea07-4da0-889f-5cdfdcd45de9,945e364e-0bc3-4ec7-a693-32fc0f8db8a1,"3300,00",2022-04-02,2022-05-01,NaN,01-30-2023,01-30-2023


### Histórico de Cargo

Para gerar a planilha com os cargos, é interessante tratar cada um deles como único, assim como acontece com cada fucionário, e gerar um ID que vai ser utilizado sempre que alguém novo entrar ou caso receba uma promoção. Por essa razão para os cargos serão geradas duas planilhas, uma contendo o ID e outra com o histórico de cada funcionário. Vale lembrar que o mesmo processo de tratamento ocorreu aqui, já que a planilha é a mesma para o histórico de salário e o trabalho que foi feito pode ser conferido no outro [jupyter notebook](tratamento_dados_salario_cargo.ipynb).

In [12]:
# Carregando a planilha que contém todos os cargos
cargos = pd.read_excel('planilhas\\cargos.xlsx')

# Criando uma coluna de senioridade, para que se caso o cargo possuir  ser preenchido nessa coluna
cargos['Senioridade'] = np.nan

# Se caso não encontrar uma senioridade, por padrão todas vão ser Pleno. 
# Criando as variáveis
pleno = 'PLENO'
pleno_sig = 'PL'
junior = 'JUNIOR'
junior_sig = 'JR'
senior = 'SENIOR'

# A lógica é, se tiver uma posição no nome do cargo, então escreva na nova coluna
for index, row in enumerate(cargos['Nome Cargo']):
    if pleno in row:
        copy_pleno = pleno
        cargos.loc[index, 'Senioridade'] = copy_pleno

    if pleno_sig in row:
        copy_pleno_sig = pleno
        cargos.loc[index, 'Senioridade'] = copy_pleno_sig

    if junior in row:
        copy_junior = junior
        cargos.loc[index, 'Senioridade'] = copy_junior

    if junior_sig in row:
        copy_junior_sig = junior
        cargos.loc[index, 'Senioridade'] = copy_junior_sig

    if senior in row:
        copy_senior = senior
        cargos.loc[index, 'Senioridade'] = copy_senior

# Aqui preenche os valores em branco com Pleno.
cargos['Senioridade'].fillna(pleno,inplace=True)

# Criando a nova planilha com o ID para cada cargo
position_sheet = pd.DataFrame({'Nome Cargo':cargos['Nome Cargo'],
                                'Senioridade': cargos['Senioridade'],
                                'Setor': cargos['Setor'],
                                'Criado_em': today,
                                'Upload_em': today})

position_sheet.insert(0,'id','')

get_uuid4(position_sheet,'Nome Cargo','id')

position_sheet.to_csv('output\\todos_cargos.csv', index=False)

position_sheet.head(10)

,id,Nome Cargo,Senioridade,Setor,Criado_em,Upload_em
0,5a1982d9-107e-411a-88b0-991113a95b8f,Auxiliar de serviços gerais,PLENO,Marketing,01-30-2023,01-30-2023
1,30926fe1-2fea-455e-b850-1051dad7016d,Montador de negativos,PLENO,TI,01-30-2023,01-30-2023
2,e605ce44-652d-439e-8389-e6accb925f2e,Agente de defesa sanitária,PLENO,RH,01-30-2023,01-30-2023
3,3af79a73-2323-428d-9b19-7bed16a2f84e,Mestre cervejeiro,PLENO,Administrativo,01-30-2023,01-30-2023
4,22c00a7b-a0d6-4907-bf80-14a1a2728210,Designer gráfico,PLENO,RH,01-30-2023,01-30-2023
5,9ae357f6-e1a0-4268-865b-88487728ec59,Marceneiro,PLENO,Vendas,01-30-2023,01-30-2023
6,2f3a9353-17a6-4722-a64c-d4e5f5539817,Jogador de vôlei,PLENO,Administrativo,01-30-2023,01-30-2023
7,52d2afff-fa2f-44bc-a6b5-fc8566f4c373,Cortador de cana-de-açucar,PLENO,Vendas,01-30-2023,01-30-2023
8,36e55d80-cc9b-407a-9f33-bb315c2a40de,Tecnólogo em negócios imobiliários,PLENO,Marketing,01-30-2023,01-30-2023
9,a3bd6354-7e02-4f97-abb4-844c9f9d1446,Ombudsman,PLENO,TI,01-30-2023,01-30-2023


Agora com a planilha com os ID's dos cargos é possível criar a planilha final. Nela vai contar a última informação para ser importada. A atribuição de cada funcionário a um ID referente ao cargo. Também todo empregado possui um gestor, o ID desse gestor deve contar em uma coluna específica, para que seja fácil do RH identificar essa imformação. Vou criar uma função exatamente igual ao ProcV do Excel, que vai procurar um valor idêntico em uma planilha e colar as demais informações em outra. Essa função e útil em outras situações, também para poder explorar mais de uma solução, além do merge do pandas.

In [13]:
def xlookup(lookup_value,
            lookup_array,
            return_array):
    '''
    Na primeira linha, estamos definindo uma função chamada xlookup com alguns argumentos.

    lookup_value: o valor que nos interessa, este será um valor de string
    lookup_array: esta é uma coluna dentro da fonte pandas dataframe,
    estamos procurando o "lookup_value" dentro desta matriz/coluna
    return_array: esta é uma coluna dentro da fonte pandas dataframe,
    queremos retornar valores desta coluna
    '''
    match_value = return_array.loc[lookup_array == lookup_value]
    if match_value.empty:
        return np.nan
    else:
        return match_value.tolist()[0]


In [14]:
# Carregando a planilha e unindo com a principal do RH.
cargos_hist = pd.read_csv('planilhas\\cargo_temp_historico.csv',sep = ',')

cargos_merge = data.merge(cargos_hist,
                            on='Nome Funcionário',
                            how='left')

#--------------------Criando CSV do Histórico de Cargo--------------------#
# Preenche as datas faltantes com a data de admissão, usando o fillna
cargos_merge['Inicial'].fillna(
cargos_merge['Dt. Admissão'],inplace=True)

# cargos_merge['Inicial'] = cargos_merge['Inicial'].astype('datetime64')

# Usando a função que defini acima. Passo primeiro a coluna onde deve conter o ID do gestor.
# E na coluna com o nome do gestor eu uso o apply para aplicar a função. Onde ela deve procurar um match 
# com o nome de todos os funcionário, se encontrar vai retornar o ID. 
cargos_merge['get_leader_id'] = cargos_merge['Gestor'].apply(xlookup, args=(
        employee_sheet['Nome'],employee_sheet['id']))


# Usando a mesma função com o mesmo critério, mas agora buscando o ID do cargo.
# Aqui vai pegar o ID do último cargo do histórico da troca.
cargos_merge['get_position_id'] = cargos_merge['Cargo'].apply(xlookup, args=(
        position_sheet['Nome Cargo'],position_sheet['id']))

# Para funcionário que não tiveram troca de cargo, vou pegar o ID do primeiro cargo. Mesma forma de utilizar a função.
cargos_merge['get_position_id_2'] = cargos_merge['Nome Cargo'].apply(xlookup, args=(
        position_sheet['Nome Cargo'],position_sheet['id']))

# Criando o CSV
positionallocation_sheet = pd.DataFrame({'Cargo_id': cargos_merge['get_position_id'],
                                        'employee_id': cargos_merge['employee_id'],
                                        'Inicial': cargos_merge['Inicial'],
                                        'Final': cargos_merge['Final'],
                                        'Vinc.Empreg.': cargos_merge['Vinc.Empreg.'],
                                        'ID_Lider': cargos_merge['get_leader_id'],
                                        'Criado_em': today,
                                        'Upload_em': today})


positionallocation_sheet.insert(0,'id','') # Criando a nova coluna com o ID

get_uuid4(positionallocation_sheet,'employee_id','id')

positionallocation_sheet.head(10)

,id,Cargo_id,employee_id,Inicial,Final,Vinc.Empreg.,ID_Lider,Criado_em,Upload_em
0,440af3f1-c1a8-43ad-a8d8-b8363119daa1,NaN,a16a156e-aa4a-40c5-94a4-4bea0f443f39,2014-05-23,NaN,CLT,4ac2c964-9996-445a-be5c-f9947fda5525,01-30-2023,01-30-2023
1,400282d4-894c-4d13-99f4-a4976b757ea5,NaN,727208ef-44e7-4c85-8c25-199f7011e9f5,1981-10-27,NaN,PJ,cdb9998c-2334-4826-a439-c6dd85808f7a,01-30-2023,01-30-2023
2,1b8f7e93-2c9a-4e88-9592-ad98e8185822,NaN,ad64f750-8850-444b-9bb1-536951a1819d,1993-06-25,NaN,CLT,3c35393c-9a82-4dee-a129-c0bc95050517,01-30-2023,01-30-2023
3,51fb966b-c383-4e9a-9882-23748f4d4bdd,NaN,945e364e-0bc3-4ec7-a693-32fc0f8db8a1,2006-02-02,2022-04-02,PJ,6e1ead10-be57-43b2-a484-32f5fa09f92a,01-30-2023,01-30-2023
4,75665c2a-a81b-4c1a-bebd-a91009e38c11,5733dca2-b17a-4386-b98d-0364a67fe49d,945e364e-0bc3-4ec7-a693-32fc0f8db8a1,2022-04-02,NaN,PJ,6e1ead10-be57-43b2-a484-32f5fa09f92a,01-30-2023,01-30-2023
5,0a835930-b4d0-43f9-bd18-dbc6da19e81e,NaN,f5fcda1f-5ef0-4e87-9768-70e048f232ae,2018-09-08,NaN,CLT,ad32fa62-15d9-4fe0-a8cf-64b047e54268,01-30-2023,01-30-2023
6,4af463ee-5178-41d0-8da2-ac3ce495adbe,NaN,9ecf4b0d-f2c6-43a6-bc78-9aec6feea1ae,1989-09-25,NaN,CLT,234f2e33-d2ed-4a13-8e03-4b915cd2e4d6,01-30-2023,01-30-2023
7,59f93e44-360c-4109-9a31-2643765984a0,NaN,9aa8a752-61db-460f-8595-1c72d2d7160b,1988-07-16,NaN,CLT,778e0a41-fe9c-411c-8439-7c46a09ef358,01-30-2023,01-30-2023
8,9b6b5142-ffde-4b18-b84e-8f082114848c,NaN,4e6cd5ab-b8dc-45b1-a1a8-d33b35f5d574,2021-02-05,NaN,PJ,c54b7014-b7e7-43fe-94a1-6da464d373d6,01-30-2023,01-30-2023
9,978fe7b9-2931-4aa8-af41-ccabb9991b08,NaN,a17c88e3-71f9-4ec8-9020-3a0fd5e23557,2021-10-26,NaN,CLT,adaaf145-cb7c-457e-807e-b5ffccdf513e,01-30-2023,01-30-2023


In [15]:
# Como é possível notar na coluna Cargo_id contém valores missing. 
# Para corrigir, agora vou preencher com o ID do primeiro cargo. Para aqueles que não receberam uma promoção e portanto, não tiveram troca. 

positionallocation_sheet['Cargo_id'].fillna(cargos_merge['get_position_id_2'], inplace=True)

# Salvando o novo CSV
positionallocation_sheet.to_csv('output\\cargos_historico.csv',index=False)

positionallocation_sheet.head(10)

,id,Cargo_id,employee_id,Inicial,Final,Vinc.Empreg.,ID_Lider,Criado_em,Upload_em
0,440af3f1-c1a8-43ad-a8d8-b8363119daa1,5a1982d9-107e-411a-88b0-991113a95b8f,a16a156e-aa4a-40c5-94a4-4bea0f443f39,2014-05-23,NaN,CLT,4ac2c964-9996-445a-be5c-f9947fda5525,01-30-2023,01-30-2023
1,400282d4-894c-4d13-99f4-a4976b757ea5,30926fe1-2fea-455e-b850-1051dad7016d,727208ef-44e7-4c85-8c25-199f7011e9f5,1981-10-27,NaN,PJ,cdb9998c-2334-4826-a439-c6dd85808f7a,01-30-2023,01-30-2023
2,1b8f7e93-2c9a-4e88-9592-ad98e8185822,e605ce44-652d-439e-8389-e6accb925f2e,ad64f750-8850-444b-9bb1-536951a1819d,1993-06-25,NaN,CLT,3c35393c-9a82-4dee-a129-c0bc95050517,01-30-2023,01-30-2023
3,51fb966b-c383-4e9a-9882-23748f4d4bdd,3af79a73-2323-428d-9b19-7bed16a2f84e,945e364e-0bc3-4ec7-a693-32fc0f8db8a1,2006-02-02,2022-04-02,PJ,6e1ead10-be57-43b2-a484-32f5fa09f92a,01-30-2023,01-30-2023
4,75665c2a-a81b-4c1a-bebd-a91009e38c11,5733dca2-b17a-4386-b98d-0364a67fe49d,945e364e-0bc3-4ec7-a693-32fc0f8db8a1,2022-04-02,NaN,PJ,6e1ead10-be57-43b2-a484-32f5fa09f92a,01-30-2023,01-30-2023
5,0a835930-b4d0-43f9-bd18-dbc6da19e81e,22c00a7b-a0d6-4907-bf80-14a1a2728210,f5fcda1f-5ef0-4e87-9768-70e048f232ae,2018-09-08,NaN,CLT,ad32fa62-15d9-4fe0-a8cf-64b047e54268,01-30-2023,01-30-2023
6,4af463ee-5178-41d0-8da2-ac3ce495adbe,9ae357f6-e1a0-4268-865b-88487728ec59,9ecf4b0d-f2c6-43a6-bc78-9aec6feea1ae,1989-09-25,NaN,CLT,234f2e33-d2ed-4a13-8e03-4b915cd2e4d6,01-30-2023,01-30-2023
7,59f93e44-360c-4109-9a31-2643765984a0,2f3a9353-17a6-4722-a64c-d4e5f5539817,9aa8a752-61db-460f-8595-1c72d2d7160b,1988-07-16,NaN,CLT,778e0a41-fe9c-411c-8439-7c46a09ef358,01-30-2023,01-30-2023
8,9b6b5142-ffde-4b18-b84e-8f082114848c,52d2afff-fa2f-44bc-a6b5-fc8566f4c373,4e6cd5ab-b8dc-45b1-a1a8-d33b35f5d574,2021-02-05,NaN,PJ,c54b7014-b7e7-43fe-94a1-6da464d373d6,01-30-2023,01-30-2023
9,978fe7b9-2931-4aa8-af41-ccabb9991b08,36e55d80-cc9b-407a-9f33-bb315c2a40de,a17c88e3-71f9-4ec8-9020-3a0fd5e23557,2021-10-26,NaN,CLT,adaaf145-cb7c-457e-807e-b5ffccdf513e,01-30-2023,01-30-2023


In [16]:
# Por fim é hora de excluir as planilhas temporárias. 

if os.path.exists('planilhas\\salario_temp_historico.csv'):
    os.remove('planilhas\\salario_temp_historico.csv')

if os.path.exists('planilhas\\cargo_temp_historico.csv'):
    os.remove('planilhas\\cargo_temp_historico.csv')
